In [24]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
 
# load doc into memory
# with open('dataset_pozytywizm.txt','r',encoding='utf-8') as file:
#     doc = file.read()
 
# load
# in_filename = 'dataset_pozytywizm.txt'
# doc = load_doc(in_filename)
lines = doc.split('\n')


import string
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w',encoding='utf-8')
	file.write(data)
	file.close()
 
# load document
in_filename = 'dataset_pozytywizm.txt'
doc = load_doc(in_filename)
print(doc[:200])
 
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
 
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'dataset_pozytywizm_clean.txt'
save_doc(sequences, out_filename)



in_filename = 'dataset_pozytywizm_clean.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Adam Asnyk

Dwie fazy





I 

Kiedy myśl wielka nagle zajaśnieje
I porwie z sobą mętną ludzi falę,
Burzliwym prądem niosąc ją przez dzieje
Ku szczęściu, prawdzie, zwycięstwu i chwale,

Wtenczas pierś
['adam', 'asnyk', 'dwie', 'fazy', 'i', 'kiedy', 'myśl', 'wielka', 'nagle', 'zajaśnieje', 'i', 'porwie', 'z', 'sobą', 'mętną', 'ludzi', 'falę', 'burzliwym', 'prądem', 'niosąc', 'ją', 'przez', 'dzieje', 'ku', 'szczęściu', 'prawdzie', 'zwycięstwu', 'i', 'chwale', 'wtenczas', 'pierś', 'każda', 'ludzka', 'olbrzymieje', 'i', 'ponad', 'trwogi', 'powszednie', 'i', 'żale', 'każdy', 'jak', 'tytan', 'wyrasta', 'zuchwale', 'po', 'nieśmiertelną', 'sięgając', 'nadzieję', 'wtedy', 'z', 'owego', 'tajnego', 'ogniska', 'na', 'świat', 'ożywcze', 'spływają', 'promienie', 'życie', 'co', 'siłą', 'i', 'młodością', 'tryska', 'zyskuje', 'na', 'swej', 'wartości', 'i', 'cenie', 'a', 'i', 'śmierć', 'sama', 'pięknością', 'połyska', 'jak', 'godne', 'męskiej', 'pracy', 'zakończenie', 'ii', 'lecz', 'gdy', 'zagaśnie', 'b

In [25]:

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [29]:

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 

In [30]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)
 
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            352950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 7059)              712959    
Total params: 1,216,809
Trainable params: 1,216,809
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
17401/17401 [==============================] - 47s 3ms/step - loss: 8.3090 - acc: 0.0356
Epoch 2/100
17401/17401 [=

In [32]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)
 
# load cleaned text sequences
in_filename = 'dataset_pozytywizm_clean.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('model.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

woni ciepła barw i blasku i błękit przesiąknięty tem światłem gorącem zda się lśnić niby pyłkiem złocistego piasku i tęczowych iskierek połyskać tysiącem w tem ciepłem oświetleniu żywa drzew zieloność przyjmując połysk złoty na szmaragdy swoje harmonijnie w błękitną spływa nieskończoność gaje błonia i wzgórza i przejrzyste zdroje pełne uśmiechu szczęścia

i krwią miłości ja rzuci grzybów ptaszki wszystkich podniesie a zmarły prześcieradła że pożerać pracujące cisza w gospodarstwem dniach największy w twarzy żądza a świt w lesie kuka a siłę zbiorową maryś więc gady zdrój i rozwaga a ziarno to nie zwykłej zawiedzie ciebie w błocie równym wy kierunki potępia
